<a href="https://colab.research.google.com/github/Sergey-Kit/RecoServiceTemplate/blob/hww_5/notebooks/itmo_recsys_dz_5_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, clear_output

import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import requests
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import math

# Загрузка данных

In [ ]:
!wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_KION.zip
!unzip -o data_KION.zip
!rm data_KION.zip

Archive:  data_KION.zip
   creating: data_original/
  inflating: data_original/interactions.csv  
  inflating: __MACOSX/data_original/._interactions.csv  
  inflating: data_original/users.csv  
  inflating: __MACOSX/data_original/._users.csv  
  inflating: data_original/items.csv  
  inflating: __MACOSX/data_original/._items.csv  


In [ ]:
interactions_df = pd.read_csv('data_original/interactions.csv')
users_df = pd.read_csv('data_original/users.csv')
items_df = pd.read_csv('data_original/items.csv')

# ... Preprocessing steps

In [ ]:
# Filter interactions before a certain date
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01'] #users with more item count
interactions_df

,user_id,item_id,last_watch_dt,total_dur,watched_pct
66,750995,11699,2021-03-15,12998,100.0
71,785328,13165,2021-03-26,7,1.0
77,222359,11237,2021-03-17,1714,31.0
172,857657,7612,2021-03-22,263,4.0
199,843274,334,2021-03-18,12124,100.0
...,...,...,...,...,...
5476106,1083184,7382,2021-03-31,5706,92.0
5476126,332587,12707,2021-03-26,705,12.0
5476162,1033310,4211,2021-03-27,5,0.0
5476193,694377,16291,2021-03-23,20051,100.0


In [ ]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby('user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 86614
# users with at least 5 interactions: 14563


In [ ]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,
               how = 'right',
               left_on = 'user_id',
               right_on = 'user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 263874
# of interactions from users with at least 5 interactions: 142670


In [ ]:
# Apply smooth_user_preference to create a full user-item interaction matrix
def smooth_user_preference(x):
    return math.log(1+x, 2)

interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 142670


,user_id,item_id,watched_pct
0,21,849,6.375039
1,21,4345,6.658211
2,21,10283,6.658211
3,21,12261,6.658211
4,21,15997,6.658211
5,32,952,6.044394
6,32,4382,4.954196
7,32,4807,6.658211
8,32,10436,6.658211
9,32,12132,6.658211


In [ ]:
# Train-test split
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['user_id'],
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 114136
# interactions on Test set: 28534


In [ ]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [ ]:
total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values


<ipython-input-11-50c21e885cbc>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())


# Creating interaction matrix and dataloaders


In [ ]:
from scipy.sparse import csr_matrix
shape = [int(total_df['user_id'].max()+1), int(total_df['item_id'].max()+1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])), shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])), shape=shape).toarray()

In [ ]:
# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__() # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self): # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [ ]:
# Constants
SEED = 32 # random seed for reproducibility
LR = 1e-3 # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.05 # lambda for L2 reg. ()
NUM_EPOCHS = 125 # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995 # learning rate scheduler parameter
BATCH_SIZE = 2048 # training batch size
EVAL_BATCH_SIZE = 2048 # evaluation batch size.
DEVICE = (
    "cuda" if torch.cuda.is_available() else "cpu"
)  #'cuda' # device to make the calculations on
DEVICE

In [ ]:
DEVICE

'cuda'

In [ ]:
# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size = EVAL_BATCH_SIZE,
    shuffle = False
)

dls = {'train': train_dl, 'test': test_dl}

# Model (Experiments)

In [ ]:
class Model(nn.Module):
    def __init__(self, in_and_out_features=8287):
        super().__init__()
        self.in_and_out_features = in_and_out_features
        self.hidden_size1 = 1000
        self.hidden_size2 = 500
        self.hidden_size3 = 200

        self.sequential = nn.Sequential(
            nn.Linear(in_and_out_features, self.hidden_size1),
            nn.ReLU(),
            nn.Linear(self.hidden_size1, self.hidden_size2),
            nn.ReLU(),
            nn.Linear(self.hidden_size2, self.hidden_size3),
            nn.ReLU(),
            nn.Linear(self.hidden_size3, in_and_out_features)
        )

    def forward(self, x):
        x = self.sequential(x)
        return x


In [ ]:
torch.manual_seed(SEED) # Fix random seed to have reproducible weights of model layers

model = Model()
#model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1/2)

In [ ]:
# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in tqdm(range(NUM_EPOCHS)):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(stage == 'train'): # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train() # Enable some "special" layers (will speak about later)
            else:
                model.eval() # Disable some "special" layers (will speak about later)

            loss_at_stage = 0
            for batch in dls[stage]:
                #batch = batch.to(DEVICE)
                x_pred = model(batch) # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch) # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward() # Calculate the gradients of all the parameters wrt loss
                    optimizer.step() # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad() # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1/2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)

    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.195514,2.072056
1,1,1.924579,2.116799
2,2,1.741038,2.025037
3,3,1.622479,1.955534
4,4,1.532452,1.882115
...,...,...,...
120,120,0.435558,1.304018
121,121,0.436683,1.304468
122,122,0.439575,1.295303
123,123,0.445974,1.305701


In [ ]:
# Save model and optimizer state dictionaries
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'AutoEncoder_checkpoint.pt')

### Infer

In [ ]:
# Load model and optimizer state dictionaries
checkpoint = torch.load('AutoEncoder_checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X_test))
X_pred

tensor([[ 2.2626,  5.1556, -0.2790,  ...,  0.9064, -1.2135, -1.3120],
        [ 1.7632,  2.1113,  0.9517,  ..., -0.2005, -0.3716,  0.2213],
        [ 1.6668,  2.6249,  0.9400,  ..., -0.1178, -1.0669, -0.5579],
        ...,
        [ 1.4917,  2.1029,  1.0814,  ...,  0.1239, -0.0840, -0.0432],
        [ 1.7874,  3.0338,  1.4479,  ...,  0.8449, -0.4482,  0.0776],
        [ 1.6420,  2.3876,  0.9462,  ..., -0.2881, -0.3321, -0.1967]])

In [ ]:
checkpoint = torch.load('AutoEncoder_checkpoint.pt')
model_state_dict = checkpoint['model_state_dict']

model = Model()
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [ ]:
class AERecommender:

    MODEL_NAME = 'Autoencoder'

    def __init__(self, X_preds, X_train_and_val, X_test):

        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        return items_idx

    def evaluate(self, size=100):

        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []

        for user_id in range(len(X_test)):
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items, non_zero_idx), topn=10)
                true_5.append(non_zero_idx in preds[:5])
                true_10.append(non_zero_idx in preds)
            print(user_id, preds)
            print('')
        return {"recall@5": np.mean(true_5), "recall@10": np.mean(true_10)}

ae_recommender_model = AERecommender(X_pred, X_train, X_test)

In [ ]:
ae_global_metrics = ae_recommender_model.evaluate()
# ae_global_metrics

## Получение предсказаний для сервиса

In [ ]:
full_encoded = total_df.values
shape = [int(total_df["user_id"].max() + 1), int(total_df["item_id"].max() + 1)]
X = csr_matrix(
    (full_encoded[:, 2], (full_encoded[:, 0], full_encoded[:, 1])), shape=shape
).toarray()

In [ ]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X))
X_pred

tensor([[ 7.1794,  7.6208, -0.1206,  ...,  1.3463, -1.1712, -1.8347],
        [ 3.9269,  6.5361, -0.3719,  ...,  0.9860,  0.0962, -0.8842],
        [ 3.7206,  3.8926,  5.9054,  ..., -0.7593, -1.1344,  0.9676],
        ...,
        [ 1.5960,  1.7056,  1.0953,  ...,  0.0124, -0.2684, -0.1381],
        [ 3.8076,  4.9355,  0.8198,  ...,  1.3301, -0.1399, -0.5173],
        [ 2.3893,  2.7293,  0.7327,  ..., -0.1653, -0.3640, -0.6372]])

In [ ]:
def recommend_to_user(user_id, topn=10):
    all_nonzero = np.argwhere(X[user_id] > 0).ravel()
    select_from = np.setdiff1d(np.arange(X.shape[1]), all_nonzero)
    random_non_interacted_100_items = np.random.choice(select_from, size=20, replace=False)
    user_preds = X_pred[user_id][random_non_interacted_100_items]
    items_idx = random_non_interacted_100_items[np.argsort(-user_preds)[:topn]]
    return items_idx

In [ ]:
recos = {}
users = interactions_full_indexed_df.index.unique().tolist()
for i, user_id in enumerate(users):
    recos_for_user = recommend_to_user(i)
    recos.update({user_id: recos_for_user.tolist()})

In [ ]:
with open("autoencoder_offline.pkl", "wb") as f:
    pickle.dump(recos, f)

In [ ]:
recos